In [2]:
import pandas as pd
# Importar utilidades para gestión de rutas
from cedenar_anomalies.utils.paths import (
    data_processed_dir,
    data_raw_dir,
    data_interim_dir,
    models_dir
)

In [4]:
user_data_filename = "cedenar_data.xlsx"
user_file = data_raw_dir(user_data_filename)
# Cargar datos usando pandas directamente
user = pd.read_excel(user_file)

In [5]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622450 entries, 0 to 622449
Data columns (total 58 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   PRODUCT_ID                  622450 non-null  int64  
 1   PRODUCTO                    622450 non-null  int64  
 2   CONTRATO                    622450 non-null  int64  
 3   CLIENTE                     622450 non-null  int64  
 4   NOMBRE_CLIENTE              622450 non-null  object 
 5   APELLIDO_CLIENTE            111432 non-null  object 
 6   ZONA                        622286 non-null  object 
 7   SECCIONAL                   622286 non-null  object 
 8   DIRECCION_PRODUCTO          622450 non-null  object 
 9   MUNICIPIO_PRODUCTO          622450 non-null  object 
 10  VEREDA_CENTRO_POBLADO       600815 non-null  object 
 11  BARRIO_PRODUCTO             600815 non-null  object 
 12  SECTOR_OPERATIVO_PRODUCTO   622450 non-null  object 
 13  COD_CICLO_CONS

In [6]:
user = user.dropna(subset=['AREA', 'PLAN_COMERCIAL','LATI_USU', 'LONG_USU', 'ZONA']).copy()
user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 561738 entries, 0 to 622448
Data columns (total 58 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   PRODUCT_ID                  561738 non-null  int64  
 1   PRODUCTO                    561738 non-null  int64  
 2   CONTRATO                    561738 non-null  int64  
 3   CLIENTE                     561738 non-null  int64  
 4   NOMBRE_CLIENTE              561738 non-null  object 
 5   APELLIDO_CLIENTE            98660 non-null   object 
 6   ZONA                        561738 non-null  object 
 7   SECCIONAL                   561738 non-null  object 
 8   DIRECCION_PRODUCTO          561738 non-null  object 
 9   MUNICIPIO_PRODUCTO          561738 non-null  object 
 10  VEREDA_CENTRO_POBLADO       543969 non-null  object 
 11  BARRIO_PRODUCTO             543969 non-null  object 
 12  SECTOR_OPERATIVO_PRODUCTO   561738 non-null  object 
 13  COD_CICLO_CONSUMO  

In [5]:
df = pd.read_csv(data_interim_dir('dataset_inference.csv'))
df.shape
# 'Usuario', 'Ejecucion', 'AREA', 'PLAN_COMERCIAL', 'Nombre','kWh Rec','cluster_id','cluster_0', 'cluster_1', 'cluster_2', 'puntaje_1', 'puntaje_2','puntaje_3', 'puntaje_4', 'puntaje_5','LATI_USU', 'LONG_USU', 'ZONA'



(768221, 16)

In [6]:
df['kWh Rec'].value_counts()

kWh Rec
0.0       28502
650.0       911
1110.0      717
2468.0      622
664.0       481
          ...  
4941.0        2
2629.0        1
3004.0        1
9348.0        1
3199.0        1
Name: count, Length: 3380, dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768221 entries, 0 to 768220
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Usuario         768221 non-null  float64
 1   Ejecucion       257588 non-null  object 
 2   AREA            768221 non-null  object 
 3   PLAN_COMERCIAL  768221 non-null  object 
 4   Nombre          250202 non-null  object 
 5   kWh Rec         207006 non-null  float64
 6   cluster_id      768221 non-null  int64  
 7   puntaje         250202 non-null  float64
 8   puntaje_1       768221 non-null  float64
 9   puntaje_2       768221 non-null  float64
 10  puntaje_3       768221 non-null  float64
 11  puntaje_4       768221 non-null  float64
 12  puntaje_5       768221 non-null  float64
 13  LATI_USU        768221 non-null  float64
 14  LONG_USU        768221 non-null  float64
 15  ZONA            768221 non-null  object 
dtypes: float64(10), int64(1), object(5)
memory usage: 93.8+ 

In [27]:
cols_sheet = ['Usuario','Ejecucion', 'AREA', 'PLAN_COMERCIAL', 'Nombre', 'kWh Rec',
               'cluster_id','puntaje','puntaje_1', 'puntaje_2','puntaje_3', 'puntaje_4', 'puntaje_5',
               'LATI_USU', 'LONG_USU', 'ZONA']
df = df.drop_duplicates(subset=cols_sheet).copy()
df.shape

(768221, 16)

In [ ]:
df = df.drop_duplicates().copy()
df = df.dropna(subset="ZONA").copy()

In [15]:
df['puntaje_pred'] = df.puntaje_pred + 1

In [17]:
df.sample(10)

,Usuario,Ejecucion,AREA,PLAN_COMERCIAL,Nombre,kWh Rec,cluster_id,cluster_0,cluster_1,cluster_2,puntaje,puntaje_pred,puntaje_1,puntaje_2,puntaje_3,puntaje_4,puntaje_5,LATI_USU,LONG_USU,ZONA
213888,1165948,2024-08-09 20:35:02-05,Urbano,31-PLAN RESIDENCIAL TELEMEDIDO,LINEA DIRECTA Y/O ACOMETIDA INTERVENIDA,1589.0,1,0.041682,0.946062,0.012256,5.0,5,0.255456,0.222300,0.027911,0.074865,0.419468,1.809204,-78.772807,PACIFICO
244480,184460,2023-03-17 13:31:20-05,Rural,2-PLAN RESIDENCIAL,FASE III,0.0,2,0.035140,0.213503,0.751357,4.0,5,0.238622,0.212321,0.039232,0.104868,0.404958,1.289402,-77.174970,CENTRO
184823,570036,2023-09-16 09:47:18-05,Urbano,2-PLAN RESIDENCIAL,INSPECCIÓN TÉCNICA CONFORME,NaN,1,0.271968,0.691754,0.036277,1.0,5,0.250061,0.225387,0.021731,0.069723,0.433097,1.786449,-78.786620,PACIFICO
157152,1067790,2022-10-03 08:12:32-05,Urbano,31-PLAN RESIDENCIAL TELEMEDIDO,LINEA DIRECTA CON CARGA CONOCIDA,0.0,1,0.055387,0.927656,0.016957,5.0,5,0.255879,0.222669,0.027958,0.073330,0.420164,1.811684,-78.769275,PACIFICO
51931,361390,2023-02-07 16:01:56-05,Urbano,2-PLAN RESIDENCIAL,SE CORRIGE ANOMALÍA,1293.0,2,0.044158,0.025356,0.930486,1.0,5,0.254923,0.229770,0.022154,0.051635,0.441518,1.670150,-77.012579,NORTE
32547,1013083,2022-06-07 17:23:22-05,Rural,2-PLAN RESIDENCIAL,LINEA DIRECTA,135.0,0,0.998705,0.000273,0.001022,5.0,5,0.254923,0.229770,0.022154,0.051635,0.441518,1.784775,-77.363008,NORTE
30378,627490,2022-05-14 19:10:34-05,Rural,2-PLAN RESIDENCIAL,ACOMETIDA INTERVENIDA O PERFORADA,NaN,0,0.991542,0.001943,0.006516,5.0,5,0.254923,0.229770,0.022154,0.051635,0.441518,1.917652,-77.315747,NORTE
174360,595602,2023-04-22 15:03:59-05,Rural,2-PLAN RESIDENCIAL,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HA,2873.0,2,0.296767,0.045123,0.658110,5.0,5,0.254923,0.229770,0.022154,0.051635,0.441518,1.695418,-78.719352,PACIFICO
15682,1221508,2024-07-19 11:39:23-05,Rural,2-PLAN RESIDENCIAL,LINEA DIRECTA,401.0,1,0.045610,0.633583,0.320807,5.0,5,0.253452,0.228444,0.022026,0.057107,0.438971,1.025467,-77.487596,SUR
144121,1198739,2022-07-19 14:02:07-05,Urbano,2-PLAN RESIDENCIAL,SE CORRIGE MEDIDOR FUERA DE SOCKET Y/O,945.0,0,0.997048,0.001461,0.001491,2.0,5,0.253009,0.228044,0.021988,0.058757,0.438203,1.804319,-78.773275,PACIFICO


In [ ]:
cluster = pd.read_csv(data_processed_dir('dataset_con_clusters.csv'))
cluster.sample(5)

In [ ]:
cluster.columns

In [ ]:
cluster_jess = pd.read_csv(data_processed_dir('df_anomaly_encoded_fzz_2_ind.csv'), encoding='latin-1')
cluster_jess.sample(5)


In [ ]:
df = cluster.copy()
df_zona = df[df['Zona'] == 'PA']

# --- Gráfico 1: Puntaje por evaluación y cluster ---
pivot = df_zona.pivot_table(index='cluster_id', columns='evaluacion', values='puntaje', aggfunc='count', fill_value=0)
# pivot = pivot[[1, 2, 3, 4, 5]] if all(i in pivot.columns for i in [1,2,3,4,5]) else pivot
# pivot.sample(5)

In [ ]:
kwh = df_zona.groupby('cluster_id')['kWh Rec'].sum().sort_index()
kwh

In [ ]:
pivot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Asume que `df` ya está cargado

# Paleta para los puntajes
colores_puntaje = {
    1: '#d6e54c',
    2: '#f97f1c',
    3: '#fbb034',
    4: '#8a851c',
    5: '#9c1c13'
}
colores = [colores_puntaje.get(i, "#cccccc") for i in sorted(colores_puntaje)]

# Zonas únicas
zonas = sorted(df['Zona'].dropna().unique())
n = len(zonas)

# Crear figura
fig, axes = plt.subplots(nrows=n, ncols=2, figsize=(14, n * 4))

for i, zona in enumerate(zonas):
    df_zona = df[df['Zona'] == zona]

    # === Gráfico 1: Desglose de puntaje por cluster ===
    pivot = df_zona.groupby('cluster_id')['puntaje'].value_counts().unstack(fill_value=0)
    pivot = pivot[[1, 2, 3, 4, 5]] if all(k in pivot.columns for k in [1, 2, 3, 4, 5]) else pivot
    ax1 = axes[i, 0]
    pivot.plot(kind='barh', stacked=True, color=colores, ax=ax1, legend=False)
    ax1.set_title(f"{zona} - Desglose Cluster por puntaje", fontsize=10)
    ax1.set_xlabel("Cantidad")
    ax1.set_ylabel("Cluster")

    # === Gráfico 2: kWh Recuperado por cluster (barh) ===
    kwh = df_zona.groupby('cluster_id')['kWh Rec'].sum().sort_index()
    ax2 = axes[i, 1]
    ax2.barh(kwh.index.astype(str), kwh.values, color='royalblue')
    ax2.set_title(f"{zona} - kWh Recuperado", fontsize=10)
    ax2.set_xlabel("kWh")
    ax2.set_ylabel("Cluster")
    # ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x / 1_000_000)} M"))

# Leyenda para los puntajes
handles = [plt.Rectangle((0, 0), 1, 1, color=colores_puntaje[k]) for k in sorted(colores_puntaje)]
labels = [f"Puntaje {k}" for k in sorted(colores_puntaje)]
fig.legend(handles, labels, title="Puntaje", bbox_to_anchor=(1.05, 0.5), loc='center left')

plt.suptitle("Fuzzy Clustering por Zona", fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0, 0.85, 0.97])
plt.show()
